In [2]:
from inner_iteration import inner_iteration
#m = inner_iteration()


In [3]:
from aerolib import *
from xfoillib import *
from exceptions import *
from inner_iteration import *

#m = inner_iteration()

In [1]:
from params import *
from geometry import *
from mass_dependencies import weigh

def log_mass():
    file = open("mass_dependencies.py", "r")
    mass_list_started = False
    for line in file.readlines():
        if line[:12] == "def weigh():":
            mass_list_started = True
            continue
        if mass_list_started:
            if line.isspace():
                continue
            line = line[4:-1]
            if line[0] == "#":
                print()
                print(line[1:])
            elif line[0] == "M" and line.split() != ["M", "=", "0"]:
                expression = " ".join(line.split()[2:]) 
                print(expression + " = ", end="" )
                exec("print(" + expression + ")")

    print("="*20)
    print(f"M = {weigh()}")
    file.close()



In [4]:
indicate_missing_param_vals = 0
show_keys_in_logs = 0
#weigh("PARAMS.csv", "GEOM.csv", indicate_missing_param_vals, show_keys_in_logs)

In [3]:
XFoil_path = "/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/"
work_path = "/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/"
#XFoil_command_CL(16, 7, 0.15, 0.8, 1.5e-5, 0, 9, 1000, XFoil_path, work_path, "SD7037")

In [4]:
'''
Модуль расчёта геометрических характеристик потребляет на вход: 
dataframe PERFORMANCE с потребными эксплуатационными характеристиками
dataframe PARAMS с параметрами электронных компонент, конструкционных материалов, аккумуляторов и т.д.
пути до файлов селиговского формата с профилем крыла wing_foil и с профилем оперения aft_foil.
TOM - значение взлётной массы в начальном приближении.

PERFORMANCE включает параметры take_off_speed, cruise_speed, flight_time, payload

PARAMS включает параметры m_FPV, m_powerplant, m_flight_control, m_fus, m_servo1, m_servo2,
line_dens_wire, line_dens_tube1, line_dens_tube2, line_dens_tube3,
area_dens_LWPLA
energy_dens_bat
number_servo1, number_servo2
l_stab
'''

# Все единицы в СИ
def inner_iteration():
    performance_file_name = "PERFORMANCE.csv"#input("имя файла performances: ")
    params_file_name = "PARAMS.csv" #input("имя файла params: ")
    settings_file_name = "SETTINGS.csv" #input("имя файла settings: ") 
    tom = 2 #input("нулевое приближение влётной массы: ")
    max_iter = 10 #input("максимальное число итераций: ")
    
    df = pd.read_csv(settings_file_name, header=None)
    tom_eps = float(dict(zip(df[0], df[1]))["tom_eps"])
    
    sculptor = Sculptor(performance_file_name, params_file_name, settings_file_name, tom)
    i = 1
    ask_about_no_value = True
    while True:
        sculptor.calculate_geometry()
        sculptor.write_info()
        pf, gf = sculptor.get_data_to_weigh()

        print(f"итерация {i}")
        new_tom = weigh(pf, gf, ask_about_no_value)
        ask_about_no_value = False
        if abs(tom - new_tom) > tom_eps:
            tom = new_tom
            sculptor.update_m(new_tom)
        else:
            print(f"сошлось на итерации: {i}")
            print("информация сохранена в файл с геометрией")
            return new_tom
        
        if i == max_iter:
            print(f"прошло {i} итераций, но расчёт всё ещё не завершён.")
            flag = 2
            while flag not in ['0', '1']:
                flag = input(f"введите 1, чтобы произвести ещё {max_iter} операций, иначе 0: ")
            if flag:
                i = 0
            else:
                print("последняя итерация геометрии сохранена")
                break
            
        i+=1
    